In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import pandas as pd
from tqdm import tqdm
import json


In [10]:
# CSV 파일 불러오기
df = pd.read_csv("hanja.csv")  # 파일 경로를 여기에 입력
df

,main_sound,level,hanja,meaning,radical,strokes,total_strokes
0,가,7급Ⅱ,家,"[[['집'], ['가']]]",宀,7,10
1,가,7급,歌,"[[['노래'], ['가']]]",欠,10,14
2,가,5급Ⅱ,價,"[[['값'], ['가']]]",人,13,15
3,가,5급,加,"[[['더할'], ['가']]]",力,3,5
4,가,5급,可,"[[['옳을'], ['가']]]",口,2,5
...,...,...,...,...,...,...,...
5973,희,특급,豨,"[[['돼지'], ['희']]]",豕,7,14
5974,희,특급,餼,"[[['보낼'], ['희']]]",食,10,19
5975,희,특급,橲,"[[['나무이름'], ['희']]]",木,12,16
5976,힐,1급,詰,"[[['꾸짖을'], ['힐']]]",言,6,13


In [11]:
# JSON 파일을 파이썬 변수로 불러오기
with open("data/text-embedding-3-small.json", "r", encoding="utf-8") as f:
    embeddings = json.load(f)

In [12]:
type(embeddings)

dict

In [13]:
len(embeddings)

6023

In [14]:
len(embeddings['哥'])

1536

In [18]:
# 'hanja'와 'radical' 컬럼만 추출
hanjas = df['hanja'].dropna().unique()
radicals = df['radical'].dropna().unique()

radicals

array(['宀', '欠', '人', '力', '口', '行', '日', '木', '貝', '車', '辵', '女', '禾',
       '艸', '衣', '馬', '疒', '玉', '言', '足', '斗', '角', '刀', '見', '肉', '門',
       '卩', '心', '殳', '竹', '目', '干', '艮', '土', '水', '手', '石', '革', '立',
       '曰', '乙', '虫', '皿', '甘', '攴', '金', '糸', '山', '戈', '龍', '冫', '酉',
       '田', '匚', '弓', '广', '阜', '米', '羊', '魚', '舟', '白', '几', '厶', '尸',
       '工', '火', '示', '虍', '廴', '巾', '黑', '鳥', '犬', '牛', '瓦', '豕', '缶',
       '八', '亠', '頁', '彳', '耒', '耳', '冂', '子', '大', '癶', '谷', '隹', '高',
       '老', '囗', '鼓', '辛', '羽', '网', '髟', '骨', '穴', '邑', '瓜', '雨', '食',
       '冖', '丨', '儿', '卜', '鬼', '丿', '矢', '匸', '臼', '一', '龜', '身', '勹',
       '毛', '韭', '麥', '厂', '止', '走', '鹿', '歹', '二', '斤', '禸', '又', '矛',
       '气', '方', '己', '月', '无', '豆', '幺', '夊', '襾', '赤', '十', '入', '而',
       '鼎', '用', '辰', '鬥', '夕', '父', '丶', '彐', '寸', '至', '韋', '毋', '片',
       '彡', '屮', '里', '戶', '黍', '鬲', '齒', '隶', '鹵', '廾', '亅', '玄', '臣',
       '麻', '豸', '氏', '面', '舛', '文', '非', '黽', '尢', '飛', '釆', '黹

In [19]:
hanjas

array(['家', '歌', '價', ..., '橲', '詰', '襭'], shape=(5978,), dtype=object)

In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm

# 한자-벡터 딕셔너리 (이미 있음: embeddings)
# 한자-부수 DataFrame (이미 있음: df)

# 1. 부수 기준 매핑
hanja_to_radical = dict(zip(df['hanja'], df['radical']))
radicals = df['radical'].dropna().unique().tolist()

# 2. 사용할 부수 리스트 (embeddings에 있는 것만)
available_radicals = [rad for rad in radicals if rad in embeddings]

# 3. 한자 목록 (embeddings에 있고 부수도 존재하는 것만)
hanjas = [h for h in df['hanja'] if h in embeddings]

# 4. Precision@K 함수
def precision_at_k(radical, k=10):
    if radical not in embeddings:
        return None

    rad_vec = np.array(embeddings[radical]).reshape(1, -1)

    hanja_vecs = []
    hanja_list = []
    for h in hanjas:
        if h == radical:  # 같은 부수는 제외
            continue
        hanja_vecs.append(embeddings[h])
        hanja_list.append(h)

    hanja_vecs = np.array(hanja_vecs)
    distances = cosine_distances(rad_vec, hanja_vecs)[0]
    sorted_indices = np.argsort(distances)
    top_k_hanjas = [hanja_list[i] for i in sorted_indices[:k]]

    true_positive = sum(hanja_to_radical.get(h) == radical for h in top_k_hanjas)
    return true_positive / k

# 5. 전체 부수 Precision@k 계산
k = 10
scores = []
for rad in tqdm(available_radicals):
    score = precision_at_k(rad, k)
    if score is not None:
        scores.append(score)
        print(f"부수 '{rad}': Precision@{k} = {score:.2f}")

# 6. 평균 정답률
mean_precision = np.mean(scores)
print(f"\n전체 부수 평균 Precision@{k}: {mean_precision:.4f}")


  0%|          | 1/213 [00:00<01:31,  2.32it/s]

부수 '宀': Precision@10 = 0.40


  1%|          | 2/213 [00:00<01:07,  3.13it/s]

부수 '欠': Precision@10 = 0.00


  1%|▏         | 3/213 [00:00<00:59,  3.53it/s]

...

부수 '支': Precision@10 = 0.00


100%|█████████▉| 212/213 [00:53<00:00,  3.82it/s]

부수 '皮': Precision@10 = 0.10


100%|██████████| 213/213 [00:53<00:00,  3.96it/s]

부수 '黃': Precision@10 = 0.00

전체 부수 평균 Precision@10: 0.1531
